# Wrangle OpenStreetMap Data [SQL]
> __UDACITY Data Analyst Nanodegree: Project 3__
<br>
__Patrick Ferry__
<br>
__December 2017__

### Table of Contents

[1. Project Objective](#po)<br>
[2. Data Preparation](#dp)<br>
[3. Process Dataset](#pd)<br>
[4. Audit and Clean Data](#acd)<br>
[5. Convert Dataset from XML to CSV](#cdxc)<br>
[6. Import CSV Files into SQL Database](#icfisd)<br>
[7. Explore Database](#ed)<br>
[8. Conclusion / Other Ideas](#oi)<br>
[9. Files](#f)<br>
[10. References](#r)<br>

<a id='po'></a>
## 1. Project Objective

Choose any area of the world from [OpenStreetMap](https://www.openstreetmap.org) and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for that part of the world. Use SQL as the data schema to complete the project.

### Skills Acquired

My goal upon completion of the case study is to improve my understanding and application of the following:

-  Assessing quality of data for validity, accuracy, completeness, consistency and uniformity
-  Parsing and gathering data from popular file formats such as .csv, .json, .xml, and .html
-  Processing data from multiple files, or very large files, that can be cleaned programmatically
-  Storing, querying, and aggregating data using SQL

### Project Overview

I chose the map area of the city of Salem, MA because that is where I grew up.  I am familiar with the area and curious to discover any anomolies or surprises since moving to NYC in 1996.

The first issue I encountered when attempting to download a map of [Salem, MA](https://www.openstreetmap.org/relation/2373035) from OpenStreetMap, was that it was last edited approximately 5 years ago.  I anticipate errors, omissions, or at the minimum outdated data.  (There doesn't seem to be a lot of interest in this area of the world, even though most people do know Salem from the historical Salem Witch Trials).

<img src="witch.jpg"> 

The second issue occurred when the export failed, perhaps due to a moved, deleted or renamed file.

Ultimately I downloaded a custom extract of the greater Salem area from [Mapzen's Metro Extracts](https://mapzen.com/data/metro-extracts/).

* <font color=orange>Map of Salem, MA:</font>
![alt text](Salem_MA.png)

### Data Overview

OpenStreetMap's data is structured in well-formed XML documents (.osm files) that consist of the following elements:

-  __Nodes__: "Nodes" are individual dots used to mark specific locations (such as a postal box). Two or more nodes are used to draw line segments or "ways".
-  __Ways__: A "way" is a line of nodes, displayed as connected line segments. "Ways" are used to create roads, paths, rivers, etc.
-  __Relations__: When "ways" or areas are linked in some way but do not represent the same physical thing, a "relation" is used to describe the larger entity they are part of. "Relations" are used to create map features, such as cycling routes, turn restrictions, and areas that are not contiguous. The multiple segments of a long way, such as an interstate or a state highway are grouped into a "relation" for that highway. Another example is a national park with several locations that are separated from each other. Those are also grouped into a "relation".

All these elements can carry tags describing the name, type of road, and other attributes.

Understanding this high level overview of the data I'd be wrangling informed me for future downstream analysis.

<a id='dp'></a>
## 2. Data Preparation

### Import libraries, packages and modules

In [5]:
import xml.etree.cElementTree as ET   # Use cElementTree or lxml if too slow
import pprint
import re
from collections import defaultdict
import csv
import codecs
import cerberus
import schema

import random
import sqlite3
import pandas as pd
from time import time
import os
from hurry.filesize import size

### OSM file preparation generating a sample data set

In the initial phases of the project I worked with a smaller, sample OSM file size to check for the most common problems to clean. The final step in the project includes queries form the larger, original OSM file.

In [2]:
# Code from Udacity Project Details section

OSM_FILE = "salem_ma.osm"
SAMPLE_FILE = "salem_sample.osm"

k = 3 # take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')
    
    # Write every k-th top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

Using the Udacity code above, the smaller sample size OSM file (“salem_sample.osm”, 16.4 MB) allows for faster processing.

<a id='pd'></a>
## 3. Process Dataset

The data exploration phase entailed familiarizing myself with the data and exploring it in order to gain a good understanding of the OpenStreetMap project in order to complete the case study.

Running the shell command “ls –l” on the original OSM file revealed the size of the file (“salem_ma.osm”, 162.4 MB), which is relevant later on when choosing an approach to parsing. Running the shell command “less” returned the meta format of the dataset, allowing for a forensic overview.

Familiarity with the dataset improved after reading through the [wiki OSM XML documentation](http://wiki.openstreetmap.org/wiki/OSM_XML) that provided a relational understanding of nodes, ways and relations and how I might work with the dataset. The data contains instances of three different data primitives (nodes, ways, relations).

Before processing the data and adding it into a database, I was interested in sorting out patterns and uniqueness. One way is to find top-level tags with key-value pairs in the OSM dataset, i.e. all distinct different types of tags (OSM, bounds, node, tag). Another is to count four other tag categories and check the k-value for each to see if there are any potential problems. A third is to find out how many unique users have contributed to the map.

Because the original file is so large, it didn't make sense to process the data by reading it into memory.  Rather than using tree-based XML parsing, which reads the entire document into memory and works with it as nodes on a tree, I chose an approach to parsing that uses a SAX parser, or iterative parsing.

Using the [iterparse()](https://docs.python.org/2/library/xml.etree.elementtree.html#xml.etree.ElementTree.iterparse) function, I looped through the dataset creating dictionaries for the above three mentioned instances:

### Tag type and count

In [3]:
# Code from Udacity Case Study [SQL] "Iterative Parsing"

def count_tags(filename):
    mytags =  defaultdict(int)
    for event, child in ET.iterparse(filename):
        if child.tag in mytags:
            mytags[child.tag] += 1
        else:
            mytags[child.tag] = 1
    
    return mytags
    
count_tags(SAMPLE_FILE)

defaultdict(int,
            {'member': 401,
             'nd': 275384,
             'node': 240142,
             'osm': 1,
             'relation': 49,
             'tag': 88171,
             'way': 30319})

<a id='pd'></a>
### Other tags, k-values and patterns

In [4]:
# Code from Udacity Case Study [SQL] "Tag Types"

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            keys["lower_colon"] += 1
        elif problemchars.search(element.attrib['k']):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    
    return keys

process_map(SAMPLE_FILE)

{'lower': 78122, 'lower_colon': 3884, 'other': 6165, 'problemchars': 0}

<a id='pd'></a>
### Unique users

In [5]:
# Code from Udacity Case Study [SQL] "Exploring Users"

def get_user(element):
    uid = element.attrib['uid']
    return uid


def process_map(filename):
    users = [] #set()
    for _, elem in ET.iterparse(filename):
        if elem.attrib.get('uid'):
            users.append(get_user(elem))
    users = set(users)
        
    return users

users = process_map(SAMPLE_FILE)
print "Number of unique users:",
print len(users)

Number of unique users: 217


<a id='acd'></a>
## 4. Audit and Clean Data

Having a good feel for how the OSM XML data was organized, and what certain tags described, was important when parsing the data.  I found that within our three data primitives (nodes, ways and relations) both nodes and ways can be tagged.

In this phase of the project there are two steps to audit and clean the data.  

First, audit the OSMFILE, using iterparse from the Element Tree module to loop through the entire dataset.  I audited 3 specific attributes (street names, postal codes, city values) and checked the values associated with each attribute to determine what needed to be cleaned.

Second, write an “update()” function to fix any incorrect data.

### Street names

In the example of ways, I used iterparse() to get the number of street types and check for problems.

In [6]:
# Code from Udacity Data Quality "Example Using Our Blueprint"

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)  # regular expression modual to parse out the street types
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] +=1
        
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)
        
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(SAMPLE_FILE):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print_sorted_dict(street_types)

print "Street types and count:"
print "\n"
audit()

Street types and count:


Ave: 3
Avenue: 60
Avenur: 1
Circle: 6
Court: 1
Drive: 12
Lane: 19
Lynnway: 1
North: 1
Place: 7
Road: 108
SalemStreet: 1
St: 2
St.: 1
street: 1
Street: 119
Terrace: 7
Way: 14


I called the “audit_street_type” function, which uses regular expression to match the last word in a street name with the street type (Ave, Blvd, St).  If the street type I found for this particular tag was incorrect, it was added to the “street_types” dictionary for unusual streets.

In [7]:
# Code from Udacity Case Study "Improving Street Names"

expected = ["Avenue", "Circle", "Court", "Drive", "Lane", "Lynnway", "North", "Place", "Road", "Square", "Street", 
            "Terrace", "Way"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)   # regular expression modual to parse out the street types
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    
    return street_types

print "Problem street names:"

audit(SAMPLE_FILE)

Problem street names:


defaultdict(set,
            {'Ave': {'Highland Ave', 'Loring Ave'},
             'Avenur': {'Harrison Avenur'},
             'SalemStreet': {'SalemStreet'},
             'St': {'Lafayette St', 'W Dane St'},
             'St.': {'Essex St.'},
             'street': {'green street'}})

Problems encountered that necessitated cleaning included street names (abbreviation, misspelling, capitalization, spacing), postal codes (uniformity, non-Salem zips) and city values (surrounding towns).

The variable 'mapping' was then changed to reflect updates needed to fix the unexpected street types to correct ones in the expected list.

In [8]:
# Code from Udacity Case Study "Improving Street Names"

mapping = { "Ave": "Avenue",
           "Avenur": "Avenue", 
           "SalemStreet": "Salem Street",
           "St": "Street",
           "St.": "Street",
           "street": "Street",
           "W": "West",
            }

def update_name(name, mapping):
    name = name.split(" ")
    for word in range(len(name)):
        if name[word] in mapping.keys():
            name[word] = mapping[name[word]]
    name = " ".join(name)
    
    return name

print "Update street names:"
print "\n"

st_types = audit(SAMPLE_FILE)
# pprint.pprint(dict(st_types))

for st_types, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping).title()
        print name, "-->", better_name

Update street names:


SalemStreet --> Salem Street
Essex St. --> Essex Street
Harrison Avenur --> Harrison Avenue
W Dane St --> West Dane Street
Lafayette St --> Lafayette Street
green street --> Green Street
Highland Ave --> Highland Avenue
Loring Ave --> Loring Avenue


Similar auditing, cleaning and updating code was applied to postal codes and city values:

### Postal codes

In [9]:
# Code adapted from Udacity "Improving Street Names"

zip_codes = []

def is_zip_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip_codes(zip_code):
    if zip_code[:3] != "019" or len(zip_code) != 5:
        zip_codes.append(zip_code)
        
def audit_zip(osmfile):
    for event, elem in ET.iterparse(SAMPLE_FILE, events=("start",)):
        if elem.tag == "node":
            for tag in elem.iter("tag"):
                if is_zip_code(tag):
                    audit_zip_codes(tag.attrib['v'])
    return zip_codes

print "Problem postal codes:"

audit_zip(SAMPLE_FILE)

Problem postal codes:


['01901-1511', '01907-2555']

In [10]:
# Code from Myles, Udacity Forum Member, in "Cleaning Postcode" Oct 2016
"""
Update pattern using regex.  Piece together regular expression pattern:
    ^ denotes match at the start
    \D represents non-digit characters
    * represents 'from zero to an infinite number'
    \d represents digit characters
    () represents the group that you want to capture
    . represents 'any character'
"""

test = ['01901-1511', '01907-2555']

def update_postcode(postcode):
    # new regular expression pattern
    search = re.match(r'^\D*(\d{5}).*', postcode)
    # select the group that is captured
    clean_postcode = search.group(1)
    return clean_postcode

print "Update postal codes:"
print "\n"

for item in test:
    cleaned = update_postcode(item)
    print cleaned

Update postal codes:


01901
01907


### City values

In [11]:
# Code from Udacity (https://discussions.udacity.com/t/need-some-help-with-update-city-name-function/262878)

expected = ["Salem"]

def audit_city(invalid_city_names, city_name):
    if city_name not in expected:
        invalid_city_names[city_name] +=1
    return invalid_city_names

def is_city_name(elem):
    return (elem.attrib['k'] == "addr:city")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    invalid_city_names = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_city_name(tag):
                    audit_city(invalid_city_names, tag.attrib['v'])
    osm_file.close()
    return invalid_city_names

print "Problem city names:"

audit(SAMPLE_FILE)

Problem city names:


defaultdict(int,
            {'Beverly': 6,
             'Danvers': 15,
             'Lynn': 10,
             'Marblehead': 1,
             'Peabody': 8,
             'Swampscott': 22})

Problem cities were not a problem nor required updating, as each accurately represents a surrounding city in the Greater Salem area.

<a id='cdxc'></a>
## 5. Convert Dataset from XML to CSV

After the initial auditing and updating was complete, the next involved preparing the data from the original OSM file to be inserted into an SQL database. This required parsing the elements in the OSM file and transforming them from document format to tabular format, thus making it possible to write to .csv files.  The .csv files are then imported into an SQL database as tables.

When parsing and cleaning the OSM file, the ‘process_map’ function passes each parent element of the OSM file to the ‘shape_element’ function.  The ‘shape_element’ function parses the information from each parent element, and its children, into dictionaries.  The dictionaries are iteratively processed by ‘process_map’ function and .csv files are generated.

*In addition to defining a schema for the .csv files and eventual tables, the code to load the data, perform iterative parsing and write the output to .csv files was provided by Udacity.*

In [12]:
# https://github.com/pratyush19/Udacity-Data-Analyst-Nanodegree/blob/master/P3-OpenStreetMap-Wrangling-with-SQL/data.py

OSM_PATH = "salem_ma.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                if child.attrib["k"] == 'addr:street':
                    node_tag["value"] = update_name(child.attrib["v"], mapping)
                elif child.attrib["k"] == 'addr:postcode':
                    node_tag["value"] = update_postcode(child.attrib["v"])
                else:
                    node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [13]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [14]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [15]:
# Note: Validation is ~ 10X slower. For the project consider using a small
# sample of the map when validating.

process_map(OSM_PATH, validate=True)
print "\n"
print "CONVERTING DATA FROM XML TO CSV:"
print "--------------------------------"
print "File Conversion Completed"



CONVERTING DATA FROM XML TO CSV:
--------------------------------
File Conversion Completed


__CSV files produced__:

In [16]:
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [17]:
filenames = find_csv_filenames("/Users/jennieferry/Desktop/pmf/udacity/Udacity II/5_Data_Wrangling/P3_Wrangle OpenStreetMap Data/revision")
for name in filenames:
    print name

nodes.csv
nodes_tags.csv
ways.csv
ways_nodes.csv
ways_tags.csv


<a id='icfisd'></a>
## 6. Import CSV Files into SQL Database

Steps*:

1. Import the modules that you need
2. Connect to the database
3. Create a cursor object
4. Create the table
5. Read in the data
6. Insert the data
7. Check that the data imported correctly
8. Close the connection

*https://discussions.udacity.com/t/creating-db-file-from-csv-files-with-non-ascii-unicode-characters/174958/7

In [18]:
# Create nodes table

sqlite_file = 'pmfdb.db'
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()
cur.execute('''CREATE TABLE nodes(id INTEGER, lat TEXT, lon TEXT, user TEXT, uid INTEGER, version TEXT, changeset TEXT, timestamp TEXT)''')
conn.commit()
with open('nodes.csv','rb') as fin:
        dr = csv.DictReader(fin) # comma is default delimiter
        to_db = [(i['id'].decode("utf-8"), i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), i['user'].decode("utf-8"), i["uid"].decode("utf-8"), i["version"].decode("utf-8"),i["changeset"].decode("utf-8"),i["timestamp"].decode("utf-8")) for i in dr]
cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
conn.commit()
conn.close()

In [19]:
# Create nodes_tags table

conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()
cur.execute('''CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)''')
conn.commit()
with open('nodes_tags.csv','rb') as fin:
        dr = csv.DictReader(fin) # comma is default delimiter
        to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()
conn.close()

In [20]:
# Create ways table

conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()
cur.execute('''CREATE TABLE ways(id INTEGER, user TEXT, uid TEXT, version TEXT, changeset TEXT, timestamp TEXT)''')
conn.commit()
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]
cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
conn.commit()
conn.close()

In [21]:
# Create ways_tags table

conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS ways_tags''')
conn.commit()
cur.execute('''CREATE TABLE ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT)''')
conn.commit()
with open('ways_tags.csv','rb') as fin:
        dr = csv.DictReader(fin) # comma is default delimiter
        to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]
cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()
conn.close()

In [22]:
# Create ways_nodes table

conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()
cur.execute('''CREATE TABLE ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)''')
conn.commit()
with open('ways_nodes.csv','rb') as fin:
        dr = csv.DictReader(fin) # comma is default delimiter
        to_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in dr]
cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
conn.commit()
conn.close()

<a id='ed'></a>
## 7. Explore Database

Equipped with cleaned .csv files and an understanding of the data inherent in those files, I set about exploring the dataset using SQL.  Queries on the data file, some top 10 instances, and other practical information about the map area helped me get a different perspective of my hometown; certain aspects I had never realized.

In [23]:
sqlite_file = 'pmfdb.db'

conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

### Data file

__Number of unique users__

In [24]:
query = '''SELECT COUNT(DISTINCT(a.uid))
FROM (SELECT DISTINCT(uid)
FROM ways
UNION ALL
SELECT DISTINCT(uid)
FROM nodes) as a;'''

results = c.execute(query)
for r in results: 
    print r

(423,)


__Number of nodes__

In [25]:
query = '''SELECT COUNT(*) FROM nodes'''

results = c.execute(query)
for r in results: 
    print r

(720425,)


__Number of ways__

In [26]:
query = '''SELECT COUNT(*) FROM ways'''

results = c.execute(query)
for r in results: 
    print r

(90956,)


### Top 10's

__Users (by number of entries)__

In [27]:
query = '''SELECT nodes_ways."user" AS "User", COUNT(*) AS "Users"
FROM (SELECT "user" FROM nodes
      UNION ALL
      SELECT "user" FROM ways) AS nodes_ways
GROUP BY nodes_ways."user"
ORDER BY "Users" DESC
LIMIT 10;'''

results = c.execute(query)

for r in results: 
    print r

(u'jremillard-massgis', 550515)
(u'morganwahl', 95983)
(u'crschmidt', 56495)
(u'MassGIS Import', 38475)
(u'Utible', 16054)
(u'wambag', 10879)
(u'Ahlzen', 10095)
(u'Alan Bragg', 6494)
(u'OceanVortex', 4449)
(u'dloutzen', 2809)


__Denominations__

In [28]:
query = '''SELECT value, count(*) as num
FROM nodes_tags
WHERE key='denomination'
GROUP BY value
ORDER BY num
DESC;'''

results = c.execute(query)
for r in results: 
    print r

(u'baptist', 9)
(u'methodist', 7)
(u'pentecostal', 3)
(u'episcopal', 2)
(u'lutheran', 2)
(u'catholic', 1)
(u'mormon', 1)


__Cuisines__

In [29]:
query = '''SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags
JOIN (SELECT DISTINCT(id)
FROM nodes_tags
WHERE value='restaurant') b
ON nodes_tags.id=b.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num
DESC limit 10;'''

results = c.execute(query)
for r in results: 
    print r

(u'pizza', 5)
(u'seafood', 2)
(u'Vegetarian,Vegan', 1)
(u'asian', 1)
(u'breakfast,_diner', 1)
(u'brunch', 1)
(u'chinese', 1)
(u'indian', 1)
(u'mexican', 1)
(u'seafood;american', 1)


__City values__

In [30]:
query = '''SELECT tags.value, COUNT(*) as count
FROM (SELECT * FROM nodes_tags
UNION ALL
SELECT * FROM ways_tags) tags
WHERE tags.key='city'
GROUP BY tags.value
ORDER BY count
DESC limit 12;'''

results = c.execute(query)
for r in results: 
    print r

(u'Swampscott', 70)
(u'Salem', 52)
(u'Peabody', 36)
(u'Lynn', 34)
(u'Danvers', 32)
(u'Beverly', 14)
(u'Marblehead', 10)


__Amenities__

In [31]:
query = '''SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key = 'amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 5;'''

results = c.execute(query)
for r in results: 
    print r

(u'school', 124)
(u'library', 53)
(u'place_of_worship', 46)
(u'bench', 28)
(u'restaurant', 26)


__Popular Streets__

In [32]:
query = '''SELECT street_names.value AS "Street", COUNT(street_names.value) AS "Times Refered"
FROM
(SELECT nodes_tags.value
FROM nodes_tags
WHERE type = 'addr' AND key = 'street'
UNION ALL
SELECT ways_tags.value
FROM ways_tags
WHERE type = 'addr' AND key = 'street'
OR
id in
(SELECT id
FROM ways_tags
WHERE key = 'highway')
AND key = 'name') AS street_names
GROUP BY street_names.value
ORDER BY "Times Refered" DESC
LIMIT 10;'''

results = c.execute(query)
for r in results: 
    print r

(u'Puritan Road', 125)
(u'Humphrey Street', 113)
(u'Stetson Avenue', 65)
(u'Salem Street', 63)
(u'Yankee Division Highway', 55)
(u'Washington Street', 44)
(u'Essex Street', 40)
(u'Nason Road', 37)
(u'Bradlee Avenue', 32)
(u'Andover Street', 29)


### Practical

__Bus stops__

In [33]:
query = '''SELECT COUNT(*)
FROM nodes_tags
WHERE value = 'bus_stop';'''

results = c.execute(query)
for r in results: 
    print r

(853,)


__Starbucks__

In [34]:
query = '''SELECT COUNT(DISTINCT(id)) 
FROM nodes_tags
WHERE value="Starbucks";'''

results = c.execute(query)
for r in results: 
    print r

(2,)


__Waterways__

In [35]:
query = '''SELECT ways_tags.value, COUNT(*) as num
FROM ways_tags
JOIN (SELECT DISTINCT(id)
FROM ways_tags
WHERE key='waterway') c
ON ways_tags.id = c.id
WHERE ways_tags.key = 'name'
GROUP BY ways_tags.value
ORDER BY num DESC;'''

results = c.execute(query)
for r in results: 
    print r

(u'Frost Fish Brook', 9)
(u'Bass River', 7)
(u'North River', 7)
(u'Crane River', 6)
(u'Breeds Pond Outlet Dam', 1)
(u'Danvers River', 1)
(u'Lynn Reservoir Dam', 1)
(u'Marblehead Boatyard', 1)
(u'Porter River', 1)
(u'Saugus River', 1)
(u'Sluice Pond Dam', 1)
(u'South River', 1)
(u'Walden Pond East End Dam', 1)
(u'Waters River', 1)


<a id='oi'></a>
## 8. Conclusion / Other Ideas

As happy as I was with the auditing and cleaning of data for Salem, MA I can't help but think Google is lightyears ahead with accurate and updated information.  I wonder if the Google Map developer site or API utilizes machine learning algorithms that audit, clean and update map information without the need of a human contributor.  The machine learning section is next.  Perhaps I'll circle back on the OSM data if there is a relevant project to test the assumption.

I'd like to see more precise custom extracts for a particular city, similar to the Metro Extracts, as surrounding areas and related data are invariablly included.  The data was relatively clean, which surprised me since it was last edited 5 years ago.  Perhaps the most frequent contributers took their responsibility and had pride in accurately reflecting as much data as possible.

Will we see an acquisition or merger between Google and OpenStreetMap?

<a id='f'></a>
## 9. Files

In [6]:
dirpath = "/Users/jennieferry/Desktop/pmf/udacity/Udacity II/5_Data_Wrangling/P3_Wrangle OpenStreetMap Data/revision"

file_size_wanted = ['parser.py',
'tags.py',
'README.md',
'sample.py',
'schema.py',
'query.py',
'database.py',
'audit.py',
'data_revised.py',
'Report_revised.ipynb',
'Wrangle_OSM_Final_revised.ipynb',
'Report_revised.pdf',
'References.html',
'Map.html',
'Report_revised.html',
'ways_tags.csv',
'ways.csv',
'nodes_tags.csv',
'ways_nodes.csv',
'salem_sample.osm',
'nodes.csv',
'pmfdb.db',
'salem_ma.osm']

files_list = []
for path, dirs, files in os.walk(dirpath):
    files_list.extend([(filename, size(os.path.getsize(os.path.join(path, filename)))) for filename in files])
    files_list.sort(key=lambda size: int(size[1].translate(None, "MKB")))  # sort numerically
    # files_list.sort(key = lambda letter: ''.join([i for i in letter[1] if not i.isdigit()]))  # sort quantifier
    
for filename, size in files_list:
    if filename in file_size_wanted:
        print '{:.<40s}: {:5s}'.format(filename,size)

README.md...............................: 1K   
sample.py...............................: 1K   
query.py................................: 2K   
schema.py...............................: 2K   
database.py.............................: 3K   
audit.py................................: 5K   
ways.csv................................: 5M   
ways_tags.csv...........................: 5M   
nodes_tags.csv..........................: 6M   
data_revised.py.........................: 7K   
ways_nodes.csv..........................: 18M  
Report_revised.ipynb....................: 31K  
salem_sample.osm........................: 52M  
Wrangle_OSM_Final_revised.ipynb.........: 59K  
nodes.csv...............................: 61M  
pmfdb.db................................: 96M  
salem_ma.osm............................: 154M 
Map.html................................: 244K 
References.html.........................: 244K 
Report_revised.html.....................: 289K 
parser.py...............................

<a id='r'></a>
## 10. References

Udacity - [https://www.udacity.com/](https://www.udacity.com/)<br>
Udacity Discussion Forum - [https://discussions.udacity.com/c/nd002-data-wrangling](https://discussions.udacity.com/c/nd002-data-wrangling)<br>
OpenStreetMap Wiki - [https://wiki.openstreetmap.org/wiki/Main_Page](https://wiki.openstreetmap.org/wiki/Main_Page)<br>
Github - [https://github.com/](https://github.com/)
 - [https://github.com/davidventuri](https://github.com/davidventuri)
 - [https://github.com/libiseller](https://github.com/libiseller)
 - [https://github.com/yudataguy](https://github.com/yudataguy)
 - [https://github.com/CassLamendola](https://github.com/CassLamendola)
 - [https://github.com/pratyush19](https://github.com/pratyush19)
 - [https://github.com/jasminej90](https://github.com/jasminej90)